In [1]:
# Installing packages
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xvzf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyarrow
try:
  # %tensorflow_version only exists in Colab.
  !pip install --disable-pip-version-check install tf-nightly
except Exception:
  pass

spark-3.1.1-bin-hadoop3.2/
spark-3.1.1-bin-hadoop3.2/NOTICE
spark-3.1.1-bin-hadoop3.2/kubernetes/
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/python_executable_check.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/autoscale.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/worker_memory_check.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/py_container_checks.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/decommissioning.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/pyfiles.py
spark-3.1.1-bin-hadoop3.2/kubernetes/tests/decommissioning_cleanup.py
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/decom.sh
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/entrypoint.sh
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/bindings/
spark-3.1.1-bin-hadoop3.2/kubernetes/dockerfiles/spark/bindings/R/
spark-3.1.1-bin-hadoop3.2/kubernetes/docker

In [2]:
# Importing necessary libraries 
import os

# setting environment paths
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import io
import pathlib

import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from pyspark.sql.functions import col, pandas_udf, regexp_extract
from tensorflow.keras.applications.imagenet_utils import decode_predictions
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, PandasUDFType
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
from PIL import Image

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


In [3]:
# Downloading CIFAR-10 dataset 
data_dir = tf.keras.utils.get_file(origin='http://pjreddie.com/media/files/cifar.tgz',
                                         fname='cifar', untar=True)

# Reading the dataset
images = spark.read.format("binaryFile").option("recursiveFileLookup", "true").option("pathGlobFilter", "*.png").load(data_dir)

168591360/168584360 [==============================] - 2s 0us/step


In [4]:
# Utility functions for feature extraction
def extract_label(path_col):
  """Extract label from file path using built-in SQL functions."""
  return regexp_extract(path_col, "_([^/.]+)", 1)

def extract_size(content):
  """Extract image size from its raw content."""
  image = Image.open(io.BytesIO(content))
  return image.size

@pandas_udf("width: int, height: int")
def extract_size_udf(content_series):
  sizes = content_series.apply(extract_size)
  return pd.DataFrame(list(sizes))

df = images.select(
  col("path"),
  col("modificationTime"),
  extract_label(col("path")).alias("label"),
  extract_size_udf(col("content")).alias("size"),
  col("content"))

In [5]:
# Listing the classes in CIFAR  10 dataset
df.select(col('label')).distinct().show()

+----------+
|     label|
+----------+
|     horse|
|      ship|
|      deer|
|       dog|
|       cat|
|  airplane|
|      bird|
|      frog|
|     truck|
|automobile|
+----------+



In [6]:
class Cifar10Dataset(Dataset):
  """
  Converts image contents into a PyTorch Dataset with standard ImageNet preprocessing.
  """
  def __init__(self, contents):
    self.contents = contents

  def __len__(self):
    return len(self.contents)

  def __getitem__(self, index):
    return self._preprocess(self.contents[index])

  def _preprocess(self, content):
    image = Image.open(io.BytesIO(content))

    # Using standard transforms for CIFAR-10 dataset
    transform = transforms.Compose([
      transforms.Resize(256),
      transforms.ToTensor(),
      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    return transform(image)

In [7]:
def cifar10_model_udf(model_fn):
  """
  Wraps an cifar model into a Pandas UDF that makes predictions.
  
  You might consider the following customizations for your own use case:
    - Tune DataLoader's batch_size and num_workers for better performance.
    - Use GPU for acceleration.
    - Change prediction types.
  """
  def predict(content_series_iter):
    model = model_fn()
    model.eval()
    for content_series in content_series_iter:
      dataset = Cifar10Dataset(list(content_series))
      loader = DataLoader(dataset, batch_size=64)
      with torch.no_grad():
        for image_batch in loader:
          predictions = model(image_batch).numpy()
          predicted_labels = [x[0] for x in decode_predictions(predictions, top=1)]
          # print(predicted_labels)
          # break
          yield pd.DataFrame(predicted_labels)
  return_type = "class: string, desc: string, score:float"
  return pandas_udf(return_type, PandasUDFType.SCALAR_ITER)(predict)

In [8]:
mobilenet_v2_udf = cifar10_model_udf(lambda: models.mobilenet_v2(pretrained=True))
predictions = df.withColumn("prediction", mobilenet_v2_udf(col("content")))
predictions.select(col("label"),col("prediction")).show(15, truncate = False)

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+-----+---------------------------------------+
|label|prediction                             |
+-----+---------------------------------------+
|frog |{n02356798, fox_squirrel, 7.472974}    |
|bird |{n02002724, black_stork, 8.031031}     |
|frog |{n01744401, rock_python, 9.125908}     |
|deer |{n02356798, fox_squirrel, 7.5615826}   |
|frog |{n02457408, three-toed_sloth, 8.139732}|
|frog |{n01756291, sidewinder, 6.9101005}     |
|bird |{n02002724, black_stork, 6.9296956}    |
|frog |{n03764736, milk_can, 9.065401}        |
|frog |{n01688243, frilled_lizard, 9.575037}  |
|frog |{n01744401, rock_python, 7.9928713}    |
|deer |{n02422106, hartebeest, 9.47989}       |
|frog |{n02119789, kit_fox, 10.59206}         |
|bird |{n02606052, rock_beauty, 6.910691}     |
|frog |{n02325366, wood_rabbit, 6.730158}     |
|frog |{n02356798, fox_squirrel, 7.484116}    |
+-----+---------------------------------------+
only showing top 15 rows



In [9]:
vgg19_udf = cifar10_model_udf(lambda: models.vgg19(pretrained=True))
predictions = df.withColumn("prediction", vgg19_udf(col("content")))
predictions.select(col("label"),col("prediction")).show(15, truncate = False)

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+-----+--------------------------------------+
|label|prediction                            |
+-----+--------------------------------------+
|frog |{n02013706, limpkin, 6.9151354}       |
|bird |{n02002724, black_stork, 7.367693}    |
|frog |{n04579145, whiskey_jug, 5.543442}    |
|deer |{n02091635, otterhound, 7.565753}     |
|frog |{n02457408, three-toed_sloth, 6.799}  |
|frog |{n02356798, fox_squirrel, 6.2871785}  |
|bird |{n02013706, limpkin, 4.2486725}       |
|frog |{n01688243, frilled_lizard, 8.424452} |
|frog |{n02356798, fox_squirrel, 6.575295}   |
|frog |{n03903868, pedestal, 6.633815}       |
|deer |{n02356798, fox_squirrel, 7.42445}    |
|frog |{n01688243, frilled_lizard, 7.2758336}|
|bird |{n06596364, comic_book, 4.652661}     |
|frog |{n02356798, fox_squirrel, 7.011245}   |
|frog |{n02356798, fox_squirrel, 7.0600677}  |
+-----+--------------------------------------+
only showing top 15 rows



In [10]:
resnet50_udf = cifar10_model_udf(lambda: models.resnet50(pretrained=True))
predictions = df.withColumn("prediction", resnet50_udf(col("content")))
predictions.select(col("label"),col("prediction")).show(15, truncate = False)

/content/spark-3.1.1-bin-hadoop3.2/python/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.
  "in the future releases. See SPARK-28264 for more details.", UserWarning)


+-----+--------------------------------------+
|label|prediction                            |
+-----+--------------------------------------+
|frog |{n02130308, cheetah, 10.426459}       |
|bird |{n01443537, goldfish, 6.834964}       |
|frog |{n01496331, electric_ray, 7.9641924}  |
|deer |{n04525038, velvet, 7.9794655}        |
|frog |{n01688243, frilled_lizard, 7.3185043}|
|frog |{n01496331, electric_ray, 7.7286572}  |
|bird |{n02356798, fox_squirrel, 5.2075057}  |
|frog |{n02356798, fox_squirrel, 8.57772}    |
|frog |{n01644900, tailed_frog, 7.5442767}   |
|frog |{n01644900, tailed_frog, 10.755041}   |
|deer |{n02115913, dhole, 10.55313}          |
|frog |{n02129165, lion, 9.034145}           |
|bird |{n02356798, fox_squirrel, 10.846921}  |
|frog |{n02356798, fox_squirrel, 9.478152}   |
|frog |{n02356798, fox_squirrel, 8.230193}   |
+-----+--------------------------------------+
only showing top 15 rows

